## `lab11`—The Weather Forecast

❖ Objectives

-   Use unit tests to verify proper code behavior.
-   Engage in pair programming exercises to build team programming skills.

<div class="alert alert-warning">
**Pair Programming**
<br />
This lab is built around *pair* programming—you need to work in pairs, although you need not be at a single machine unless you prefer to work that way.  At the end, when you report collaborators, please report the names and NetIDs of all partners in this lab exercise.  (In exceptional cases, such as the room layout, trios are permitted.)
</div>

The big picture:  we have a series of Python scripts which do the following:

1.  `grab_stations` retrieves a list of National Weather Service stations and prints the station call sign and latitude/longitude.
2.  `grab_forecast` accepts the foregoing data and retrieves the current temperature or a forecast, adding this as a column to a similar output.
3.  `plot_forecast` plots the resulting temperatures against a map of the state of Illinois.

You are going to determine how to compose scripts 1 and 2, and then work together as a time to implement the full toolchain.

The lab will take place in three parts:

1.  The TA will teach a brief lesson covering how the pieces of the toolchain work and what you need to implement.
2.  You and the TA will write appropriate unit tests which your code will have to pass.  You may share code in class using [this Etherpad](https://public.etherpad-mozilla.org/p/Pa6X2Bzq0h), but please be careful in using it as a common resource.
3.  The class will divide into two groups (pairs).  One partner will write the `grab_stations` script, while the other will write the `grab_forecast` script.
2.  Once your code works, you and your partner will put your pieces together to make the end result work.

### Working on the Command Line

We've worked in the Jupyter notebook almost exclusively for the labs, but this time we're going to run some code directly on the command line.  Open a Terminal window to work in, and make sure that you're in the right directory.  (You shouldn't type the `$`—it's there to remind you that the code is at the prompt rather than in Python.)
    
    $ cd ; cd cs101-sp16/lab11

One of the current Python scripts can be executed by simply telling Python to run a file (instead of to start the interpreter):
    
    $ python3 grab_stations.pyc

<div class="alert alert-info">
We're doing something a little funny here—note that `pyc` extension instead of `py`.  This is a *compiled* Python file, which means that Python has stored the script in a faster intermediate code that isn't human-readable anymore.  (These get replaced if you edit and run the script, so they're never a problem.)  Since you need to *write* the scripts but still need to *test* the toolchain, we provide these `pyc` files as a substitute for testing against.
</div>

To run the entire toolchain, the command line lets you take the *output* from one script and use it as the *input* to another.  Thus:
    
    $ python3 grab_stations.pyc | python3 grab_forecast.pyc | python3 plot_forecast.py

This takes a moment to run and then displays a map of Illinois with the stations and today's forecast high temperatures visible.

If you are in charge of writing `grab_forecast.py`, then you can use `grab_stations.pyc` to test your code and make sure it works:
    
    $ python grab_stations.pyc | python grab_forecast.py

### Accessing Web Data:  `requests`

Programmers and scientists often need to access data which are not located directly on the hard drive, so `open` won't take care of it.  If the data are available on the web, we can use the `requests` library to access the server, grab the data, and then parse it as a single string (just like `read()` for a file).

This example grabs a web page and outputs the HTML markup code underlying the page:

In [ ]:
import requests

url = 'http://www.nws.noaa.gov/mdl/gfslamp/lavlamp.shtml'
page = requests.get(url)

print(page.text)

This example grabs data from an online source:

In [ ]:
url = 'https://raw.githubusercontent.com/davis68/cs101-example/master/exoplanets.csv'
planets = requests.get(url)

print(planets.text)

Part of this lab will involve scraping data from a web page.  This means that you have to pull a large chunk of data (the web page as a string) and then fish around to find the exact datum you are looking for.

### Reading Input from the Command Line

If you need to receive input from another program, a straightforward way to do this is by piping the commands together.  This way, the output of the first program becomes the input of the second.

In today's case, you simply need to know how to turn the *standard input stream*, or `stdin` into a string you can parse.

In [ ]:
# this won't show anything in Jupyter because there's no `stdin`, but here's what you would do:
import sys
for line in sys.stdin:
    print(line)

### Unit Tests

Unit tests are simple claims about the way the code should behave.  Since they are essentially probes of how well your code works, we'll write some sensible tests first for the components of the scripts you will write.

-   First, we will all work together to discuss what the two programs should do and how they should talk to each other.
-   Next, we will all work together to write unit tests for validating correct behavior.
-   Then we will split up into the pairs to write code that satisfies our unit tests.

At this point you should divide into pairs (there should be at most ONE group of three).  One partner will be in charge of writing `grab_stations.py` and the other will be in charge of `grab_forecast.py`.  The following will assist you in this process.

Unit tests should be written in the format:
    
    def test_function_name(input):    # where function_name is the name of the original function we're testing
        test_result = function_name(whatever)
        assert test_result == actual_result

###  `grab_stations.py`

If you are on this team, you'll need to open the skeleton code file `grab_stations.py`.  (That's where you should work for this exercise, and will be part of your submission.)  The program has the structure:

    import requests

    def grab_website_data():
        '''Get raw data as HTML string from the NOAA website.'''
        url = 'http://www.nws.noaa.gov/mdl/gfslamp/docs/stations_info.shtml'
        pass  # YOUR CODE HERE

    def extract_section(text):
        '''Find Illinois data segment (in a PRE tag).
        We know (from examination) that inside of the PRE block containing ' IL '
        (with whitespace and case matching) we can find the IL station data.
        This solution isn't robust, but it's good enough for practical cases.'''
        il_start  = text.find(' IL ')
        tag_start = text.rfind('PRE', il_start-200, il_start) # look backwards
        tag_end   = text.find('PRE', il_start)
        return text[tag_start+4:tag_end-2]

    def parse_station_line(line):
        '''Extract latitude and longitude of stations. We know the columns are fixed
        (which is both inconvenient and convenient). In this case, we will simply
        set the limits of the relevant columns by counting the number of columns
        over we need to go.'''
        pass  # YOUR CODE HERE


    text = grab_website_data()
    data = extract_section(text)
    for line in data.splitlines():
        try:
            stn, lat, lon = parse_station_line(line)
            print('%s\t%f\t%f'%(stn,lon,lat))
        except:
            print('Could not parse line\n\t%s'%line)

To test:
    
    $ python3 grab_stations.py | python3 grab_forecast.pyc

or
    
    $ python3 grab_stations.py | python3 grab_forecast.pyc | python3 plot_forecast.py

### `grab_forecast.py`

If you are on this team, you'll need to open the skeleton code file `grab_forecast.py`.  (That's where you should work for this exercise, and will be part of your submission.)  The program has the structure:

    def grab_stdin(text=sys.stdin):
        '''Get input stations from stdin.'''
        stns = []
        locx = []
        locy = []
        for line in text:
            try:
                pass  # YOUR CODE HERE
            except:
                print('Could not parse line \n\t"%s"'%line)
        return stns, locx, locy

    def grab_forecast_data():
        '''Get raw data as HTML string from the NOAA website.'''
        url = 'http://www.nws.noaa.gov/mdl/gfslamp/lavlamp.shtml'
        pass  # YOUR CODE HERE

    def get_station_temp(temp_data, stn):
        '''We have a list of Illinois stations from the sites loaded previously.
           We need to load the data for each of those sites and store these data 
           locally.  There are a lot of data included here, but we are only 
           interested in one:  the current temperature, located at the index
           offset 169 and of length 2 (found by examination).
        '''
        tag_start = temp_data.find(stn)
        if tag_start == -1:
            T = float('NaN')
            return
        tag_end = tag_start + 1720 #each text block is 1720 characters long
        T = float(temp_data[tag_start+169:tag_start+172])
        return T
    
    stns, locx, locy = grab_stdin()
    temp_data = grab_forecast_data()
    for stn,lat,lon in zip(stns, locx, locy):
        temp = get_station_temp(temp_data, stn)
        print('%s\t%f\t%f\t%f'%(stn,lon,lat,temp))

To test:
    
    $ python3 grab_stations.pyc | python3 grab_forecast.py

or
    
    $ python3 grab_stations.pyc | python3 grab_forecast.py | python3 plot_forecast.py

### Putting It All Together

Now you and your partner need to collaborate to make these codes work together.  There's not a really easy way to share the files directly with the tools available to you in CS 101, so we suggest that one of you email the script to the other so you can get them both in the same folder.  (This will, of course, overwrite the remaining skeleton code file in that folder.)
    
    $ python3 grab_stations.py | python3 grab_forecast.py | python3 plot_forecast.pyc

If this doesn't work, debug and figure out where things went wrong, and fix them.  You've got this!

### Stretch Goal

If you make it this far, awesome!  Try this out if there is more than twenty minutes left:  randomly pair with a different person in the lab and swap code with them.  If you both wrote to the unit tests, then this should work fine—if not, try to decide if the flaw is with the unit tests (called *test coverage*) or with the way you wrote the code to adhere to those standards.

When reporting your work, please use the following format.  Mark which script you wrote with a star.
    
    *grab_stations.py:  netid1
    grab_forecast.py:  netid2

Both of you should submit your work normally (making sure that your files and folders are in the proper directory `lab11/`).